<a href="https://colab.research.google.com/github/2003Yash/CAPSTONE_PROJECT/blob/main/CAPSTONE_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotional Latent Space Mapping for Sentiment Prediction
# Using Llama 3-70B model from Groq

In [ ]:
# Install required libraries
!pip install groq umap-learn scikit-learn plotly numpy pandas matplotlib sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.cluster import KMeans
import umap
import json
import re
from typing import List
from sentence_transformers import SentenceTransformer
from groq import Groq

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API')

In [ ]:

# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

# Function to get embeddings using Llama 3-70B from Groq
def get_llm_embedding(text: str, model="llama3-70b-8192") -> List[float]:
    """Generate embeddings using Llama 3-70B model from Groq."""
    try:
        # Create a system prompt asking the model to return a JSON embedding vector
        system_prompt = """You are an embedding model. Return a JSON array containing a 768-dimensional vector
        that represents the semantic meaning of the input text. The vector should be a list of floating point
        numbers. Return only the JSON array, with no other explanation or text."""

        # Make API call to Groq
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Generate an embedding for this text: {text}"}
            ],
            temperature=0.0,
            max_tokens=1600,
        )

        # Extract the embedding from the response
        embedding_text = response.choices[0].message.content

        # Extract the JSON array from the response using regex
        match = re.search(r'\[.*\]', embedding_text, re.DOTALL)
        if match:
            embedding_json = match.group(0)
            embedding = json.loads(embedding_json)

            # Ensure we have a consistent embedding size
            if len(embedding) > 768:
                embedding = embedding[:768]
            elif len(embedding) < 768:
                # Pad with zeros if necessary
                embedding = embedding + [0.0] * (768 - len(embedding))

            return embedding
        else:
            raise ValueError("Could not extract embedding from response")
    except Exception as e:
        print(f"Error getting embeddings: {e}")
        # Return a random vector as fallback
        return np.random.uniform(-0.1, 0.1, 768).tolist()

In [ ]:

# Function to get embeddings using SentenceTransformer (faster and more reliable)
def get_sentence_transformer_embedding(text: str) -> List[float]:
    """Generate embeddings using SentenceTransformer model."""
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding = model.encode([text])[0]
    return embedding.tolist()

# Function to generate embeddings for a list of texts
def get_embeddings_batch(texts: List[str], use_llm=False) -> np.ndarray:
    """Generate embeddings for a list of texts."""
    embeddings = []
    for i, text in enumerate(texts):
        if i % 5 == 0:
            print(f"Processing text {i+1}/{len(texts)}")

        if use_llm:
            embedding = get_llm_embedding(text)
        else:
            embedding = get_sentence_transformer_embedding(text)
        embeddings.append(embedding)

        # Add a slight delay to avoid overwhelming the API
        if use_llm and i < len(texts) - 1:
            import time
            time.sleep(1)

    return np.array(embeddings)

In [ ]:

# Function to reduce dimensionality using UMAP
def reduce_dimensions_umap(embeddings: np.ndarray, n_components=3) -> np.ndarray:
    """Reduce dimensions using UMAP."""
    reducer = umap.UMAP(n_components=n_components, random_state=42)
    reduced = reducer.fit_transform(embeddings)
    return reduced

# Function to cluster the embeddings
def cluster_embeddings(reduced_embeddings: np.ndarray, n_clusters=5) -> np.ndarray:
    """Cluster the reduced embeddings using KMeans."""
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(reduced_embeddings)
    return clusters


In [ ]:

# Function to visualize the clusters in 3D
def visualize_3d_clusters(reduced_embeddings: np.ndarray, clusters: np.ndarray, texts: List[str], title='3D Emotional Latent Space') -> None:
    """Visualize clusters in 3D using Plotly."""
    df = pd.DataFrame({
        'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'z': reduced_embeddings[:, 2],
        'cluster': clusters,
        'text': texts
    })

    fig = px.scatter_3d(
        df, x='x', y='y', z='z',
        color='cluster',
        hover_data=['text'],
        title=title,
        labels={'cluster': 'Emotional Cluster'},
        color_continuous_scale=px.colors.qualitative.Bold
    )

    fig.update_traces(marker=dict(size=6),
                      selector=dict(mode='markers'))

    fig.update_layout(
        scene=dict(
            xaxis_title='Dimension 1',
            yaxis_title='Dimension 2',
            zaxis_title='Dimension 3'
        ),
        width=900,
        height=700
    )

    fig.show()

In [ ]:


# Function to visualize the clusters in 2D
def visualize_2d_clusters(reduced_embeddings: np.ndarray, clusters: np.ndarray, texts: List[str], title='2D Emotional Latent Space') -> None:
    """Visualize clusters in 2D using Plotly."""
    df = pd.DataFrame({
        'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'cluster': clusters,
        'text': texts
    })

    fig = px.scatter(
        df, x='x', y='y',
        color='cluster',
        hover_data=['text'],
        title=title,
        labels={'cluster': 'Emotional Cluster'},
        color_continuous_scale=px.colors.qualitative.Bold
    )

    fig.update_traces(marker=dict(size=10),
                      selector=dict(mode='markers'))

    fig.update_layout(
        xaxis_title='Dimension 1',
        yaxis_title='Dimension 2',
        width=900,
        height=700
    )

    fig.show()

In [ ]:

# Function to track emotional trajectory
def track_emotional_trajectory(user_messages: List[str], reduced_embeddings: np.ndarray, user_ids: List[str]) -> None:
    """Track and visualize a user's emotional trajectory."""
    unique_users = set(user_ids)

    for user in unique_users:
        user_indices = [i for i, uid in enumerate(user_ids) if uid == user]
        if len(user_indices) < 2:
            continue

        user_trajectory = reduced_embeddings[user_indices]
        user_messages_subset = [user_messages[i] for i in user_indices]

        # Create a DataFrame for this user's trajectory
        df = pd.DataFrame({
            'x': user_trajectory[:, 0],
            'y': user_trajectory[:, 1],
            'z': user_trajectory[:, 2] if user_trajectory.shape[1] > 2 else np.zeros(len(user_indices)),
            'message': user_messages_subset,
            'sequence': range(1, len(user_indices) + 1)
        })

        # Create a 3D plot if we have 3D data
        if user_trajectory.shape[1] > 2:
            fig = px.line_3d(
                df, x='x', y='y', z='z',
                hover_data=['message', 'sequence'],
                title=f'Emotional Trajectory for User {user}',
                markers=True,
                color_discrete_sequence=['#FF5733']
                # line=dict(width=5)  # Remove or adjust this line
            )

            # Apply line width using update_traces if needed
            fig.update_traces(line=dict(width=5))  # Add this line

            # Add text labels for the sequence numbers
            fig.add_trace(
                go.Scatter3d(
                    x=df['x'],
                    y=df['y'],
                    z=df['z'],
                    mode='markers+text',
                    text=df['sequence'],
                    textposition='top center',
                    hoverinfo='none',
                    showlegend=False,
                    marker=dict(size=8, symbol='circle', color='#FF5733')
                )
            )

            fig.update_layout(
                scene=dict(
                    xaxis_title='Dimension 1',
                    yaxis_title='Dimension 2',
                    zaxis_title='Dimension 3'
                ),
                width=900,
                height=700
            )

            fig.show()

In [ ]:

# Comprehensive sample data with more emotional nuance
sample_feedback = [
    # Positive emotions
    "I'm really happy with your service, everything worked perfectly!",
    "This product is amazing, exceeded all my expectations.",
    "Wow, I'm impressed by how quickly you resolved my issue!",
    "I'm excited to try the new features you just released!",
    "I'm pleased with the improvements in the latest update.",
    "I'm grateful for the exceptional customer service I received today.",
    "The team went above and beyond to help me, I'm truly delighted!",
    "I feel valued as a customer when you respond this quickly.",
    "Your product brings me joy every time I use it.",
    "I'm thrilled with how intuitive your new interface is.",

    # Neutral emotions
    "I'm somewhat satisfied, but there's room for improvement.",
    "The customer service was okay, but I had to wait a long time.",
    "I'm feeling neutral about this product, it works but nothing special.",
    "It's what I expected, neither impressive nor disappointing.",
    "I can see both pros and cons with this service.",
    "The product seems average compared to others in the market.",
    "I'm neither satisfied nor dissatisfied with my purchase.",
    "The features work as advertised, nothing more nothing less.",
    "It gets the job done, but I wouldn't specifically recommend it.",
    "This update didn't really change my experience either way.",

    # Negative emotions
    "I'm not happy with how this was handled, very disappointing.",
    "This is absolutely terrible, I want my money back!",
    "I'm furious about how my complaint was ignored for weeks!",
    "I'm frustrated with the constant bugs in your software.",
    "I'm a bit disappointed but understand these things happen.",
    "I'm angry that I paid premium prices for subpar quality.",
    "I feel cheated by the misleading advertising of this product.",
    "Your customer service makes me want to switch to a competitor.",
    "I regret purchasing this product, it's been nothing but problems.",
    "I'm upset about how complicated your return process is.",

    # Confused emotions
    "I'm confused about how to use this feature, not very intuitive.",
    "I don't understand why you changed the interface, it's puzzling.",
    "I'm not sure if this product is right for me anymore.",
    "I'm perplexed by the contradicting information on your website.",
    "I can't figure out how to cancel my subscription, it's baffling.",
    "The instructions are confusing and difficult to follow.",
    "I'm uncertain whether I should upgrade or stay with my current plan.",
    "I'm bewildered by the number of options available.",
    "The pricing structure is confusing me; I don't know which tier to choose.",
    "I'm trying to understand why my account was charged twice.",

    # Surprised emotions
    "I'm surprised by how well this works compared to competitors.",
    "I didn't expect such a quick response to my inquiry!",
    "The quality is surprisingly better than I anticipated.",
    "I'm shocked that you managed to fix this issue so quickly.",
    "The package arrived much sooner than expected, what a pleasant surprise!",
    "I was taken aback by how many features are included in the basic plan.",
    "It's astounding how much value you provide for the price.",
    "I'm amazed at the level of personalization in your service.",
    "The unexpected bonus in my order was a wonderful surprise.",
    "I never thought customer service could be this responsive!"
]

# Sample user conversation sequences - each representing a different emotional journey
user_conversations = {
    "user1": [
        "Hi, I'm interested in your premium plan.",
        "The features look good, but I'm concerned about the price.",
        "Actually, this is too expensive for me.",
        "Your competitor offers the same for less, I'll go there instead."
    ],
    "user2": [
        "Hello, I'm having trouble logging in.",
        "Thanks for the quick response.",
        "That fixed it! You guys are awesome.",
        "I'll definitely recommend your service to others."
    ],
    "user3": [
        "My account was charged incorrectly.",
        "I've been waiting for 3 days with no response.",
        "This is unacceptable, I want to speak to a manager!",
        "I'm going to report this to consumer protection."
    ],
    "user4": [
        "I'm confused about how to use the new dashboard.",
        "Thanks for the explanation, that helps a bit.",
        "I'm still having trouble with some features.",
        "Ok, now I get it. The interface is actually quite clever!"
    ],
    "user5": [
        "Just bought your product and excited to try it!",
        "Hmm, having some issues with the installation.",
        "Support was helpful but still can't get it working right.",
        "Finally working! Not as easy as advertised but good now."
    ]
}


In [ ]:

# Main function to process sample data and generate visualizations
def main():
    print("Running Emotional Latent Space Mapping with Llama 3-70B...")

    # Step 1: Process sample feedback data
    print("\n--- Processing sample feedback data ---")

    # Get embeddings for sample feedback - defaulting to SentenceTransformer for speed
    print("Generating embeddings for sample feedback...")
    embeddings = get_embeddings_batch(sample_feedback, use_llm=False)

    # Reduce dimensions for visualization
    print("Reducing dimensions using UMAP...")
    reduced_3d = reduce_dimensions_umap(embeddings, n_components=3)
    reduced_2d = reduce_dimensions_umap(embeddings, n_components=2)

    # Cluster the embeddings
    print("Clustering the embeddings...")
    clusters = cluster_embeddings(reduced_3d, n_clusters=5)

    # Visualize the clusters
    print("Visualizing the emotional clusters...")
    visualize_3d_clusters(reduced_3d, clusters, sample_feedback, "3D Emotional Latent Space - Customer Feedback")
    visualize_2d_clusters(reduced_2d, clusters, sample_feedback, "2D Emotional Latent Space - Customer Feedback")

    # Step 2: Process user conversations and track emotional trajectories
    print("\n--- Processing user conversation trajectories ---")

    # Combine all user conversations
    all_messages = []
    user_ids = []

    for user_id, messages in user_conversations.items():
        all_messages.extend(messages)
        user_ids.extend([user_id] * len(messages))

    # Get embeddings for all messages
    print("Generating embeddings for user conversations...")
    trajectory_embeddings = get_embeddings_batch(all_messages,)

    # Reduce dimensions for visualization
    print("Reducing dimensions for trajectories...")
    trajectory_reduced = reduce_dimensions_umap(trajectory_embeddings, n_components=3)

    # Plot all trajectories in the same space
    print("Visualizing all emotional trajectories together...")

    # Create a combined visualization of all trajectories
    df_all = pd.DataFrame({
        'x': trajectory_reduced[:, 0],
        'y': trajectory_reduced[:, 1],
        'z': trajectory_reduced[:, 2],
        'user_id': user_ids,
        'message': all_messages,
        'message_index': [i+1 for user in user_conversations.values() for i in range(len(user))]
    })

    fig = px.line_3d(
        df_all, x='x', y='y', z='z',
        color='user_id',
        line_group='user_id',
        hover_data=['message', 'message_index'],
        title='Combined Emotional Trajectories',
        labels={'user_id': 'User ID'},
        markers=True
    )

    fig.update_traces(marker=dict(size=8),
                     line=dict(width=4))

    fig.update_layout(
        scene=dict(
            xaxis_title='Dimension 1',
            yaxis_title='Dimension 2',
            zaxis_title='Dimension 3'
        ),
        width=900,
        height=700
    )

    fig.show()

    # Track individual emotional trajectories
    print("Tracking individual emotional trajectories...")
    track_emotional_trajectory(all_messages, trajectory_reduced, user_ids)

    print("\nEmotional Latent Space Mapping completed successfully!")


In [ ]:

# Entry point - run the main function
if __name__ == "__main__":
    # Import here to prevent circular import
    import plotly.graph_objects as go

    # Execute the main function
    main()

Running Emotional Latent Space Mapping with Llama 3-70B...

--- Processing sample feedback data ---
Generating embeddings for sample feedback...
Processing text 1/50
Processing text 6/50
Processing text 11/50
Processing text 16/50
Processing text 21/50
Processing text 26/50
Processing text 31/50
Processing text 36/50
Processing text 41/50
Processing text 46/50
Reducing dimensions using UMAP...
Clustering the embeddings...
Visualizing the emotional clusters...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.




--- Processing user conversation trajectories ---
Generating embeddings for user conversations...
Processing text 1/20
Processing text 6/20
Processing text 11/20
Processing text 16/20
Reducing dimensions for trajectories...
Visualizing all emotional trajectories together...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



Tracking individual emotional trajectories...



Emotional Latent Space Mapping completed successfully!


In [ ]:
# 📈 What This Upgrade Does:

# Feature	Old Code	                      ---------> New Upgraded Code
# Only Groq usage	❌ (Mixed)              ---------> 	✅ (Only Groq Llama 3-70B)
# Batch processing	❌ (One by one)        --------->	✅ (Async batch using asyncio)
# More clustering options	❌ (Only KMeans) ---------> 	✅ (KMeans + DBSCAN optional)
# Save embeddings to files	❌             --------->	✅ CSV and JSON saving
# Trajectory tracking	Limited               --------->	✅ Animated line plots
# Error handling	Basic	                   ----------> ✅ Resilient fallback
# Text audit for embeddings	❌            --------->	✅ Save raw records

In [ ]:
# Emotional Latent Space Mapping using Groq (Llama 3-70B)

# Install dependencies
!pip install groq umap-learn scikit-learn plotly numpy pandas matplotlib

In [ ]:
# Imports
import os
import time
import json
import re
import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from typing import List, Dict
from sklearn.cluster import KMeans, DBSCAN
import umap
from groq import Groq

In [ ]:

# Function: Get embedding from Groq
async def get_llm_embedding_async(text: str, model="llama3-70b-8192") -> List[float]:
    system_prompt = """You are an embedding model. Return a pure JSON array of exactly 768 floats
    representing the semantic meaning of the provided input text."""

    try:
        response = await client.chat.completions.acreate(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Text: {text}"}
            ],
            temperature=0.0,
            max_tokens=1600
        )
        content = response.choices[0].message.content
        match = re.search(r'\[.*\]', content, re.DOTALL)
        if match:
            embedding_json = match.group(0)
            embedding = json.loads(embedding_json)
            # Normalize length
            if len(embedding) > 768:
                embedding = embedding[:768]
            elif len(embedding) < 768:
                embedding += [0.0] * (768 - len(embedding))
            return embedding
    except Exception as e:
        print(f"Embedding Error: {e}")
        return np.random.uniform(-0.1, 0.1, 768).tolist()

In [ ]:

# Function: Batch embeddings
async def get_embeddings_batch(texts: List[str]) -> np.ndarray:
    tasks = [get_llm_embedding_async(text) for text in texts]
    embeddings = await asyncio.gather(*tasks)
    return np.array(embeddings)

In [ ]:

# Function: UMAP Reduction
def reduce_dimensions(embeddings: np.ndarray, n_components=3) -> np.ndarray:
    reducer = umap.UMAP(n_components=n_components, random_state=42)
    reduced = reducer.fit_transform(embeddings)
    return reduced

In [ ]:

# Function: Clustering
def cluster_embeddings(reduced_embeddings: np.ndarray, method="kmeans", n_clusters=5) -> np.ndarray:
    if method == "kmeans":
        model = KMeans(n_clusters=n_clusters, random_state=42)
    elif method == "dbscan":
        model = DBSCAN(eps=0.5, min_samples=5)
    else:
        raise ValueError("Unknown clustering method")
    labels = model.fit_predict(reduced_embeddings)
    return labels

In [ ]:

# Function: Save Embeddings
def save_embeddings(embeddings: np.ndarray, texts: List[str], clusters: np.ndarray, filename_prefix: str):
    df = pd.DataFrame(embeddings)
    df['text'] = texts
    df['cluster'] = clusters
    df.to_csv(f"{filename_prefix}_embeddings.csv", index=False)
    df.to_json(f"{filename_prefix}_embeddings.json", orient="records")

In [ ]:

# Function: 3D Visualization
def visualize_clusters_3d(reduced_embeddings: np.ndarray, clusters: np.ndarray, texts: List[str], title="3D Emotional Space"):
    df = pd.DataFrame({
        'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'z': reduced_embeddings[:, 2],
        'cluster': clusters,
        'text': texts
    })
    fig = px.scatter_3d(
        df, x='x', y='y', z='z', color='cluster',
        hover_data=['text'], title=title,
        color_continuous_scale=px.colors.qualitative.Bold
    )
    fig.update_layout(width=900, height=700)
    fig.show()

In [ ]:

# Function: 2D Visualization
def visualize_clusters_2d(reduced_embeddings: np.ndarray, clusters: np.ndarray, texts: List[str], title="2D Emotional Space"):
    df = pd.DataFrame({
        'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'cluster': clusters,
        'text': texts
    })
    fig = px.scatter(
        df, x='x', y='y', color='cluster',
        hover_data=['text'], title=title,
        color_continuous_scale=px.colors.qualitative.Bold
    )
    fig.update_layout(width=900, height=700)
    fig.show()

In [ ]:

# Function: Track Emotional Trajectory (Animated)
def track_emotional_trajectory(reduced_embeddings: np.ndarray, texts: List[str], user_ids: List[str]):
    df = pd.DataFrame({
        'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'user_id': user_ids,
        'text': texts,
        'step': range(len(texts))
    })
    fig = px.line(
        df, x='x', y='y', color='user_id', line_group='user_id',
        hover_data=['text', 'step'],
        title="User Emotional Trajectory"
    )
    fig.update_layout(width=900, height=700)
    fig.show()


In [ ]:

# Example: Sample Data
sample_texts = [
    "I'm thrilled with the update!",
    "I'm confused about the settings.",
    "This service disappointed me.",
    "I'm surprised by the fast delivery!",
    "Neutral feelings about the support team.",
    "Frustrated by recurring bugs.",
    "Very happy with my experience!",
    "I'm angry about the billing issue.",
    "I'm amazed by the new features!",
    "Uncertain if I want to continue subscription."
]

sample_user_ids = ["user1", "user2", "user1", "user3", "user2", "user1", "user3", "user2", "user1", "user3"]


In [ ]:

# Main Workflow
async def main_workflow(texts: List[str], user_ids: List[str]):
    embeddings = await get_embeddings_batch(texts)
    reduced = reduce_dimensions(embeddings, n_components=3)
    clusters = cluster_embeddings(reduced, method="kmeans", n_clusters=5)
    save_embeddings(reduced, texts, clusters, "emotional_mapping")
    visualize_clusters_3d(reduced, clusters, texts)
    visualize_clusters_2d(reduced, clusters, texts)
    track_emotional_trajectory(reduced, texts, user_ids)


In [ ]:
# Run Main
await main_workflow(sample_texts, sample_user_ids)

Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'
Embedding Error: 'Completions' object has no attribute 'acreate'


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:2462: UserWarning:

n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1

